<a href="https://colab.research.google.com/github/keithy1012/MachineLearning/blob/main/GaussianMixtureModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
class GaussianMixtureModel:
    def __init__(self, n_components, max_iter=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.weights = None
        self.means = None
        self.covariances = None

    def initialize_parameters(self, X):
      self.weights = np.full(self.n_components, 1/self.n_components)
      self.means = X[np.random.choice(X.shape[0], self.n_components, replace=False)]
      self.covariances = [np.cov(X.T) for _ in range(self.n_components)]

    def gaussian_model(self, X, mean, covariance):
      covariance_det = np.linalg.det(covariance)
      covariance_inv = np.linalg.inv(covariance)
      exponent = -0.5 * np.sum((X - mean) @ covariance_inv * (X - mean), axis=1)
      return (1 / np.sqrt(2 * np.pi * covariance_det)) * np.exp(exponent)

    def expectation_step(self, X):
      probability = np.zeros((X.shape[0], self.n_components))
      for i in range(self.n_components):
        probability[:, i] = self.weights[i] * self.gaussian_model(X, self.means[i], self.covariances[i])

      probability /= probability.sum(axis=1, keepdims=True)
      return probability

    def maximization_step(self, X):
      probability = self.expectation_step(X)
      self.weights = probability.mean(axis=0)
      self.means = (X.T @ probability / self.weights).T

      for i in range(self.n_components):
        diff = X - self.means[i]
        self.covariances[i] = diff.T @ np.diag(probability[:, i]) @ diff / self.weights[i]

    def fit(self, X):
      self.initialize_parameters(X)
      for _ in range(self.max_iter):
        old_means = self.means.copy()
        old_weights = self.weights.copy()
        old_covariances = self.covariances.copy()
        self.maximization_step(X)
        if (np.linalg.norm((np.array(self.means) - np.array(old_means)).tolist()) < self.tol) and (np.linalg.norm((np.array(self.weights) - np.array(old_weights)).tolist()) < self.tol) and (np.linalg.norm((np.array(self.covariances) - np.array(old_covariances)).tolist()) < self.tol):
          break

      return self.weights, self.means, self.covariances

    def predict(self, X):
      probability = self.expectation_step(X)
      return probability, np.argmax(probability, axis=1)


In [ ]:
components = 3
gmm = GaussianMixtureModel(n_components=components)

X = np.array([
    [0.5, 1.2],
    [1.8, 2.4],
    [-0.6, 0.8],
    [3.2, 5.1],
    [3.8, 5.5],
    [-2.1, 4.0],
    [-2.5, 3.8],
    [0.1, -0.2],
    [3.0, 4.8],
    [-1.8, 4.2],
    [2.9, 5.0],
    [-0.5, 0.6],
    [3.5, 5.3],
    [-2.0, 3.9],
    [0.7, 1.0]
]) * 100
weights, means, covariances = gmm.fit(X)

for i in range(components):
    print(f"Component {i+1}:")
    print(f"  Weight: {weights[i]}")
    print(f"  Mean: {means[i]}")
    print(f"  Covariance:")
    print(covariances[i])
    print()



probs, max = gmm.predict(np.array([[1,2], [3,4]]))
print(probs)
print(max)

Component 1:
  Weight: 0.36600916853534826
  Mean: [1000. 4740.]
  Covariance:
[[1.37684e+07 6.21823e+07]
 [6.21823e+07 2.94128e+08]]

Component 2:
  Weight: 0.43244111414772785
  Mean: [1000. 4740.]
  Covariance:
[[1.37684e+07 6.21823e+07]
 [6.21823e+07 2.94128e+08]]

Component 3:
  Weight: 0.2015497173169239
  Mean: [1000. 4740.]
  Covariance:
[[1.37684e+07 6.21823e+07]
 [6.21823e+07 2.94128e+08]]

1
1
